In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

from buffer import AllActivationBuffer
from trainers.scae import TrainerConfig
from training import train_scae_suite
from utils import load_model_with_folded_ln2

from datasets import load_dataset
import torch as t
from nnsight import LanguageModel
from collections import defaultdict
from tqdm import tqdm

DTYPE = t.bfloat16
device = "cuda:0" if t.cuda.is_available() else "cpu"

model = load_model_with_folded_ln2("gpt2", device=device, torch_dtype=DTYPE)

dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    '''dumb helper class to make the dataset iterable'''
    def __init__(self, dataset):
        self.data = iter(dataset)
    def __iter__(self):
        return self
    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
C = 10
expansion = 16
k = 128

num_features = model.config.n_embd * expansion
n_layer = model.config.n_layer

In [9]:
pretrained_configs = {}
connections = defaultdict(dict)

for down_layer in range(n_layer):
    for module in ['attn', 'mlp']:
        down_name = f'{module}_{down_layer}'
        pretrained_configs[f'{module}_{down_layer}'] = {
            'repo_id': 'jacobcd52/scae', 
            'filename': f'ae_{module}_{down_layer}.pt',
            'k' : k
            }
        
        # Use random connections for testing
        if module=='mlp':
            for up_layer in range(down_layer+1): # mlp sees attn from same layer
                up_name = f'{module}_{up_layer}'
                connections[down_name][up_name] = t.randint(0, num_features, (num_features, C), dtype=t.long)

In [10]:
initial_submodule = model.transformer.h[0]
submodules = {}
for layer in range(n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")
    
buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    initial_submodule=initial_submodule,
    d_submodule=model.config.n_embd,
    n_ctxs=128,
    out_batch_size = 32,
    refresh_batch_size = 256,
    device=device,
    dtype=DTYPE,
)

/root/dictionary_learning/notebooks/../buffer.py:182: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with t.cuda.amp.autocast(dtype=self.dtype):
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
trainer_cfg = TrainerConfig(
    connection_sparsity_coeff=0.0,
    steps=10,
)

trainer = train_scae_suite(
    buffer,
    module_specs=pretrained_configs,
    trainer_config=trainer_cfg,
    connections=connections,
    steps=100,
    save_steps = 10,
    dtype = DTYPE,
    device=device,
    # save_dir: Optional[str] = None,
    # log_steps: Optional[int] = None,
    # use_wandb: bool = False,
    # hf_repo_id: Optional[str] = None,
    # seed: Optional[int] = None,
)

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
suite = trainer.suite
def run_evaluation(
        suite, 
        buffer, 
        n_batches=10, 
        ce_batch_size=32,
        use_sparse_connections=False
        ):
    '''Simple function to run evaluation on several batches, and return the average metrics'''
    
    varexp_metrics = {name : {} for name in buffer.submodules.keys()}
    ce_metrics = {name : {} for name in buffer.submodules.keys()}

    for i in tqdm(range(n_batches)):
        # get varexp metrics
        input_acts, output_acts = next(buffer)
        batch_varexp_metrics = suite.evaluate_varexp_batch(
            input_acts, 
            output_acts,
            use_sparse_connections=use_sparse_connections
            )

        # get CE metrics
        b = buffer.refresh_batch_size
        buffer.refresh_batch_size = ce_batch_size
        tokens = buffer.token_batch()
        batch_ce_metrics = suite.evaluate_ce_batch(
            model, 
            tokens, 
            buffer.submodules,
            use_sparse_connections=use_sparse_connections
            )
        buffer.refresh_batch_size = b

        for name in ce_metrics.keys():
            for metric in batch_ce_metrics[name].keys():
                ce_metrics[name][metric] = ce_metrics[name].get(metric, 0) + batch_ce_metrics[name][metric] / n_batches
            for metric in batch_varexp_metrics[name].keys():
                varexp_metrics[name][metric] = varexp_metrics[name].get(metric, 0) + batch_varexp_metrics[name][metric] / n_batches
           
    return varexp_metrics, ce_metrics

In [12]:
varexp_metrics, ce_metrics = run_evaluation(
    suite, 
    buffer, 
    n_batches=10, 
    ce_batch_size=32,
    use_sparse_connections=False
    )

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


ReferenceError: weakly-referenced object no longer exists